In [10]:
from internal import *
import torch
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    preprocess_image

num_classes = 2
model_main = DenseNet121(num_classes,pretrained=True)
checkpoint0 = torch.load("Model_densenet121_state.pth")

def reshape_transform(tensor, height=14, width=14):
    # print(tensor.shape)
    # result = tensor[:, 1:, :].reshape(tensor.size(0),
    #                                   height, width, tensor.size(2))

    # Bring the channels to the first dimension,
    # like in CNNs.
    result = tensor.transpose(2, 3).transpose(1, 2)
    return result

layer = [model_main.encoder4]

cam = GradCAM(model=model_main,
            target_layers=layer,
            reshape_transform=reshape_transform)
COVID = f"data/training/COVID/"
NO_COVID = f"data/training/NonCOVID/"

image_path = f"{COVID}2020.01.24.919183-p27-132.png"
rgb_img = preprocess(image_path)[:, :, ::-1]
rgb_img = cv2.resize(rgb_img, (224, 224))
rgb_img = np.float32(rgb_img) / 255

input_tensor = preprocess_image(rgb_img, mean=[0.5, 0.5, 0.5],
                                    std=[0.5, 0.5, 0.5])

# Otherwise, targets the requested category.
targets = None

# AblationCAM and ScoreCAM have batched implementations.
# You can override the internal batch size for faster computation.
cam.batch_size = 32

grayscale_cam = cam(input_tensor=input_tensor,
                    targets=targets)

# Here grayscale_cam has only one image in the batch
grayscale_cam = grayscale_cam[0, :]

cam_image = show_cam_on_image(rgb_img, grayscale_cam)
cv2.imwrite(f'test_cam.jpg', cam_image)

# COVID_PATH = f"data/training/COVID/"
# NO_COVID_PATH = f"data/training/NonCOVID/"
# image_path1 = f"{COVID_PATH}2020.01.24.919183-p27-132.png"
# image_path2 = f"{NO_COVID_PATH}30_0.jpg"
# print(infer_many([image_path1, image_path2]))
# # print(infer_single(image_path2))

True